In [ ]:
!pip install pypdf langchain sentence-transformers openai transformers chromadb tiktoken

In [ ]:
# Step 1:
#from langchain.document_loaders import TextLoader # for text files
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
# Step 2:
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.tools import Tool
# Step 3:
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.prompts import MessagesPlaceholder
from langchain_core.messages import SystemMessage
# Step 4:
from langchain.agents import AgentExecutor

# Step 0:
Set some hyperparameters for preprocessing and models later:

In [ ]:
import os
# Step 1
chunk_size = 256
chunk_overlap = 20
embedding_model = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
k_retriever = 5
# Step 2
os.environ["OPENAI_API_KEY"] = ""

# Step 1:
Read the documents and save them to a temporary chroma database.

In [ ]:
def preprocess_documents_return_retriever():
  # get pdfs
  report_path = "Beispieltext.pdf"
  loader = PyPDFLoader(report_path)
  documents = loader.load()
  # it is possible to add different loaders - e.g. web-scraping etc.

  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)

  # load embedding model from disk
  embeddings = SentenceTransformerEmbeddings(model_name=embedding_model,  model_kwargs = {'device': 'cuda'})

  # create embeddings in chroma
  db = Chroma.from_documents(docs, embeddings)

  # create retriever to retrieve most relevant documents from chroma store and return it
  retriever = db.as_retriever(k=k_retriever)
  return retriever

# Step 2:
Initialize chat model (ChatGPT) and create retriever tool.

In [ ]:
def initialize_and_return_chatgpt_plus_tools(retriever):
  chat_model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-1106")

  # use retriever from step 1
  retrieve_tool = create_retriever_tool(retriever=retriever, name="bekomm_zusatzinformation", description="nützlich um zusätzliche Information zu Raiffeisen, Wien, Linz und Falco zu bekommen.")

  tools = [retrieve_tool]
  return chat_model, tools

# Step 3:
Create components for RAG agent. Following components are necessary:
*   The memory
*   The prompt template
*   The agent
*   The agent executor

In [ ]:
def create_components_and_return_agent_and_memory(llm, tools):
  # this is needed for both the memory and the prompt
  memory_key = "history"

  memory = AgentTokenBufferMemory(memory_key=memory_key, llm=llm)

  # changed to German
  system_message = SystemMessage(
    content=(
        "Sei immer höflich und beantworte die Fragen mit bestem Gewissen. "
        "Wenn du die Antwort nicht weißt, antworte mit 'Ich weiß die Antwort zu deiner Frage leider nicht.' "
        "Versuche die Frage erst mit deinem eigenen Wissen zu beantworten. Wenn du die Antwort so nicht weißt, dann erst benutze ein Tool um die relevanten Informationen zu bekommen."
    )
  )
  # put together the prompt for ChatGPT
  prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)],
  )

  agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
  return agent, memory

# Step 4:
Initialize agent executor.


In [ ]:
def create_and_return_agent_executor(agent, tools, memory):
  agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    return_intermediate_steps=True,
  )
  return agent_executor

# Step 5:
try out agent

In [ ]:
retriever = preprocess_documents_return_retriever()

chat_model, tools = initialize_and_return_chatgpt_plus_tools(retriever)

agent, memory = create_components_and_return_agent_and_memory(chat_model, tools)

agent_executor = create_and_return_agent_executor(agent, tools, memory)

In [ ]:
result = agent_executor({"input": "Was kannst du mir über Wien erzählen?"})



> Entering new AgentExecutor chain...

Invoking: `bekomm_zusatzinformation` with `{'query': 'Wien'}`


[Document(page_content='Wien (Aussprache [viːn]) anhören ⓘ/? ist die Bundeshauptstadt der Republik Österreich und zugleich \neines der neun österreichischen Bundesländer. Mit mehr als 2 Millionen Einwohnern (Oktober', metadata={'page': 2, 'source': 'Beispieltext.pdf'}), Document(page_content='Wien (Aussprache [viːn]) anhören ⓘ/? ist die Bundeshauptstadt der Republik Österreich und zugleich \neines der neun österreichischen Bundesländer. Mit mehr als 2 Millionen Einwohnern (Oktober', metadata={'page': 2, 'source': 'Beispieltext.pdf'}), Document(page_content='bezeichnet. Wien ist eine Großstadt mit sehr hoher Lebensqualität.  \nBezeichnung  \nDer Name der Stadt wird stan darddeutsch [vi:n] ausgesprochen. Im bairisch -österreichischen Dialekt', metadata={'page': 2, 'source': 'Beispieltext.pdf'}), Document(page_content='bezeichnet. Wien ist eine Großstadt mit sehr hoher Lebensqualität. 